In [1]:
!pip install transformers
!pip install sentencepiece

In [3]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

In [4]:
from transformers import AutoModelForMaskedLM

from transformers import AutoTokenizer, AutoModel
import re

tokenizer4 = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
tokenizer5 = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [5]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [6]:
import numpy as np
import pandas as pd

In [7]:
import random
seed_val=3407
torch.manual_seed(seed_val)
np.random.seed(seed_val)
random.seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [8]:
# Load whole datasets
df_train = pd.read_csv("/kaggle/input/ipo-wholedataset/IPO_WholeDataset_ClassCreate.csv", names = ["tweet", "classes"])

In [9]:
df_train = df_train.iloc[1: , :]
df_train.head()

,tweet,classes
1,Millions of Indians investing in the country’s...,0.0
2,టోకు ద్రవ్యోల్బణమే కీలకం via తాజా వార్తలు | ...,1.0
3,મેરે રે કો એસે ધક ધક હો રહા હે....\nfingers cr...,1.0
4,The country's largest insurer will list itself...,1.0
5,Can a full time employee ask for board of dire...,1.0


In [10]:
df_train.describe()

,tweet,classes
count,960,960
unique,944,3
top,Kaun kaun god se prey kar rahe ki unko LICIPO ...,1.0
freq,3,657


In [11]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
LEARNING_RATE = 1e-08

In [12]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.tweet
        #self.targets = self.data.class
        # using loop
        #y=list(df_train['class'])
        y=list(dataframe.classes)
        res = [eval(i) for i in y]
        print("Modified list is: ", res)
        yInt = [int(res) for res in res]
        print("final->",yInt)
        self.targets = yInt
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
            #'targets': torch.tensor(self.targets[index], dtype=torch.str)
        }

In [13]:
train_size = 0.6
train_data=df_train.sample(frac=train_size,random_state=seed_val)
test_data=df_train.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(df_train.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set4 = SentimentData(train_data, tokenizer4, MAX_LEN)
testing_set4 = SentimentData(test_data, tokenizer4, MAX_LEN)

training_set5 = SentimentData(train_data, tokenizer5, MAX_LEN)
testing_set5 = SentimentData(test_data, tokenizer5, MAX_LEN)

FULL Dataset: (960, 2)
TRAIN Dataset: (576, 2)
TEST Dataset: (384, 2)
Modified list is:  [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 2.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 2.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 

In [14]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader4 = DataLoader(training_set4, **train_params)
testing_loader4 = DataLoader(testing_set4, **test_params)

training_loader5 = DataLoader(training_set5, **train_params)
testing_loader5 = DataLoader(testing_set5, **test_params)


In [15]:
weights = [.762, .316, .923]
class_weights=torch.tensor(weights,dtype=torch.float)
loss_function = torch.nn.CrossEntropyLoss(weight = class_weights)

In [16]:
class mBERTClass(torch.nn.Module):
    def __init__(self):
        super(mBERTClass, self).__init__()
        self.l1 = AutoModelForMaskedLM.from_pretrained("bert-base-multilingual-cased")
        #self.pre_classifier = torch.nn.Linear(768, 768)
        self.pre_classifier = torch.nn.Linear(119547, 68)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(68, 3)
        self.loss_fn = loss_function

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [17]:
class XLMRobertaClass(torch.nn.Module):
    def __init__(self):
        super(XLMRobertaClass, self).__init__()
        self.l1 = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")
        #self.pre_classifier = torch.nn.Linear(768, 768)
        self.pre_classifier = torch.nn.Linear(250002, 68)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(68, 3)
        self.loss_fn = loss_function

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [18]:
model4 = mBERTClass()
model4.to(device)

model5 = XLMRobertaClass()
model5.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


XLMRobertaClass(
  (l1): XLMRobertaForMaskedLM(
    (roberta): XLMRobertaModel(
      (embeddings): XLMRobertaEmbeddings(
        (word_embeddings): Embedding(250002, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): XLMRobertaEncoder(
        (layer): ModuleList(
          (0): XLMRobertaLayer(
            (attention): XLMRobertaAttention(
              (self): XLMRobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): XLMRobertaSelfOutput(
                (dense)

In [19]:
# Creating optimizer
optimizer4 = torch.optim.SGD(params =  model4.parameters(), lr=LEARNING_RATE)
optimizer5 = torch.optim.SGD(params =  model5.parameters(), lr=LEARNING_RATE)

In [20]:
# For Model 4 , pair 1 to be updated
predicted14 = []
original14=[]
# For Model 5 , pair 1 to be updated
predicted15 = []
original15=[]

def calcuate_accuracy(preds, targets, number):
    if number==1:
        predicted11.append(preds.tolist())
        original11.append(targets.tolist())
        n_correct1 = (preds==targets).sum().item()
        return n_correct1
    if number==2:
        predicted12.append(preds.tolist())
        original12.append(targets.tolist())
        n_correct2 = (preds==targets).sum().item()
        return n_correct2
    if number==3:
        predicted13.append(preds.tolist())
        original13.append(targets.tolist())
        n_correct3 = (preds==targets).sum().item()
        return n_correct3
    if number==4:
        predicted14.append(preds.tolist())
        original14.append(targets.tolist())
        n_correct4 = (preds==targets).sum().item()
        return n_correct4
    if number==5:
        predicted15.append(preds.tolist())
        original15.append(targets.tolist())
        n_correct5 = (preds==targets).sum().item()
        return n_correct5

In [21]:
def calcuate_accuracyTrain(preds, targets):
    n_correct = (preds==targets).sum().item()
    #print("Inside accuracyTrain() method")
    return n_correct

In [22]:
def train(model, optimizer, training_loader, epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracyTrain(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            #print(f"Training Loss per 5000 steps: {loss_step}")
            #print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [23]:
EPOCHS = 4
print(f"Finetuning with FOURTH model:::")
for epoch in range(EPOCHS):
    train(model4, optimizer4, training_loader4, epoch)
print(f"Finetuning with FIFTH model:::")
for epoch in range(EPOCHS):
    train(model5, optimizer5, training_loader5, epoch) 

Finetuning with FOURTH model:::


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2345: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
72it [00:26,  2.71it/s]


The Total Accuracy for Epoch 0: 45.65972222222222
Training Loss Epoch: 1.1988943612409964
Training Accuracy Epoch: 45.65972222222222


72it [00:25,  2.86it/s]


The Total Accuracy for Epoch 1: 45.833333333333336
Training Loss Epoch: 1.2196904598838754
Training Accuracy Epoch: 45.833333333333336


...

The Total Accuracy for Epoch 1: 39.93055555555556
Training Loss Epoch: 2.4744975889722505
Training Accuracy Epoch: 39.93055555555556


72it [00:37,  1.93it/s]


The Total Accuracy for Epoch 2: 44.96527777777778
Training Loss Epoch: 2.42523864739471
Training Accuracy Epoch: 44.96527777777778


72it [00:37,  1.93it/s]

The Total Accuracy for Epoch 3: 44.791666666666664
Training Loss Epoch: 2.0818223588996463
Training Accuracy Epoch: 44.791666666666664


In [24]:
def valid(model, testing_loader, number):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accuracy(big_idx, targets, number)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                #print(f"Validation Loss per 100 steps: {loss_step}")
                #print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu

In [25]:
acc4 = valid(model4, testing_loader4, 4)
print(f"Validation Accuracy FOURTH Model::::\n")
print("Accuracy on test data = %0.2f%%" % acc4)
acc5 = valid(model5, testing_loader5, 5)
print(f"Validation Accuracy FIFTH Model::::\n")
print("Accuracy on test data = %0.2f%%" % acc5)

96it [00:06, 15.99it/s]


Validation Loss Epoch: 1.063129232575496
Validation Accuracy Epoch: 52.083333333333336
Validation Accuracy FOURTH Model::::

Accuracy on test data = 52.08%


96it [00:08, 10.94it/s]

Validation Loss Epoch: 1.2266359538771212
Validation Accuracy Epoch: 66.92708333333333
Validation Accuracy FIFTH Model::::

Accuracy on test data = 66.93%


In [26]:
print("***********************************************")
print(len(predicted14))
print(predicted14)
print("***********************************************")
print(len(predicted15))
print(predicted15)

***********************************************
96
[[1, 1, 1, 0], [0, 0, 1, 1], [1, 0, 1, 1], [1, 0, 1, 1], [1, 0, 0, 0], [0, 1, 1, 1], [1, 1, 1, 0], [1, 1, 0, 0], [0, 0, 1, 1], [1, 0, 0, 1], [1, 1, 1, 1], [1, 1, 1, 1], [0, 0, 0, 1], [0, 1, 1, 1], [0, 1, 0, 0], [0, 1, 0, 0], [1, 1, 1, 0], [1, 1, 0, 0], [1, 1, 0, 1], [1, 0, 1, 0], [1, 0, 1, 1], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 1, 1, 1], [1, 0, 1, 1], [1, 0, 1, 1], [0, 0, 0, 0], [1, 1, 1, 0], [0, 0, 1, 1], [0, 1, 1, 1], [0, 1, 1, 1], [1, 0, 0, 1], [1, 0, 1, 1], [1, 0, 0, 0], [1, 0, 0, 1], [0, 0, 0, 1], [1, 1, 0, 0], [1, 1, 0, 1], [1, 1, 1, 0], [1, 1, 0, 0], [1, 0, 1, 1], [1, 0, 1, 1], [1, 1, 0, 1], [1, 0, 0, 1], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 1, 1], [1, 0, 1, 1], [1, 1, 0, 1], [1, 1, 1, 1], [1, 1, 1, 1], [0, 1, 0, 0], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [0, 0, 1, 0], [1, 1, 1, 0], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 0, 0], [1, 1, 0, 1], [1, 1, 0, 1], [1, 0, 1, 0], [1, 0, 0, 1], [1, 1, 1, 1], [1, 1, 1, 

In [27]:
print("**************************************")
print(len(original14))
print(original14)
print("**************************************")
print(len(original15))
print(original15)


**************************************
96
[[0, 1, 2, 1], [1, 0, 2, 0], [1, 0, 1, 1], [0, 1, 1, 1], [1, 1, 1, 2], [1, 1, 1, 0], [1, 1, 1, 0], [2, 0, 1, 0], [2, 0, 1, 0], [0, 1, 1, 0], [1, 1, 1, 0], [0, 0, 1, 1], [1, 1, 0, 1], [1, 1, 0, 1], [0, 0, 1, 1], [1, 0, 1, 0], [0, 0, 0, 1], [0, 0, 0, 1], [1, 1, 1, 1], [0, 1, 1, 0], [0, 1, 2, 1], [1, 0, 1, 0], [1, 1, 1, 1], [0, 1, 1, 1], [0, 2, 1, 1], [1, 1, 0, 1], [1, 0, 0, 1], [1, 0, 1, 1], [1, 1, 1, 0], [1, 1, 1, 1], [0, 1, 1, 0], [1, 1, 0, 0], [0, 0, 0, 1], [2, 1, 1, 1], [1, 0, 2, 1], [1, 1, 0, 1], [0, 2, 1, 1], [1, 1, 1, 1], [1, 1, 1, 2], [1, 1, 1, 0], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [2, 1, 1, 2], [1, 1, 1, 1], [1, 1, 1, 0], [1, 1, 1, 1], [1, 1, 1, 0], [1, 1, 1, 1], [0, 0, 0, 1], [1, 2, 0, 1], [1, 1, 1, 1], [0, 0, 1, 1], [2, 1, 2, 0], [1, 2, 1, 0], [1, 1, 0, 0], [1, 0, 1, 1], [0, 1, 1, 1], [1, 1, 2, 2], [0, 1, 1, 2], [1, 1, 1, 1], [1, 1, 2, 1], [1, 1, 1, 2], [2, 1, 1, 1], [1, 1, 1, 1], [1, 0, 1, 0], [1, 2, 1, 1], [2, 1, 1, 0], [1, 1

In [28]:
final_prediction4 = []
final_prediction5 = []
for sublist in predicted14:
    for item in sublist:
        final_prediction4.append(item)
for sublist in predicted15:
    for item in sublist:
        final_prediction5.append(item)

In [29]:
final_original4 = []
final_original5 = []
for sublist in original14:
    for item in sublist:
        final_original4.append(item)
for sublist in original15:
    for item in sublist:
        final_original5.append(item)

In [30]:
print("*****************************************")
print(len(final_prediction4))
print(final_prediction4)
print("*****************************************")
print(len(final_prediction5))
print(final_prediction5)

*****************************************
384
[1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,

In [31]:
print("**************************")
print(len(final_original4))
print(final_original4)
print("**************************")
print(len(final_original5))
print(final_original5)

**************************
384
[0, 1, 2, 1, 1, 0, 2, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 0, 2, 0, 1, 0, 2, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 2, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 0, 2, 1, 1, 1, 0, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 2, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 2, 1, 2, 0, 1, 2, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 2, 2, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 2, 1, 1, 2, 1, 1, 0, 1, 1, 0, 2, 1, 1, 1, 2, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 2, 1, 2, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,

In [32]:
import sklearn

In [33]:
print("***********Confusion Matrix by MODEL 4 over Test Set**********************")
mat4 = sklearn.metrics.confusion_matrix(final_original4,final_prediction4)
mat4

***********Confusion Matrix by MODEL 4 over Test Set**********************


array([[ 40,  53,   0],
       [ 98, 160,   0],
       [  9,  24,   0]])

In [34]:
print("***********Confusion Matrix by MODEL 5 over Test Set**********************")
mat5 = sklearn.metrics.confusion_matrix(final_original5,final_prediction5)
mat5

***********Confusion Matrix by MODEL 5 over Test Set**********************


array([[  0,  93,   0],
       [  1, 257,   0],
       [  1,  32,   0]])

In [35]:
print("***********Accuracy by MODEL 4 over Test Set**********************")
sklearn.metrics.accuracy_score(final_original4,final_prediction4)

***********Accuracy by MODEL 4 over Test Set**********************


0.5208333333333334

In [36]:
print("***********Accuracy by MODEL 5 over Test Set**********************")
sklearn.metrics.accuracy_score(final_original5,final_prediction5)

***********Accuracy by MODEL 5 over Test Set**********************


0.6692708333333334

In [38]:
print(len(final_prediction4))
with open("final_prediction4_S2_U.txt", "w") as output:
    output.write(str(final_prediction4))

384


In [39]:
print(len(final_prediction5))
with open("final_prediction5_S2_U.txt", "w") as output:
    output.write(str(final_prediction5))

384
